# Importing Modules and loading Dataset

In [1]:
__author__ = 'pratyush'
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pickle


In [46]:
#data_Naviga1 = json.load(open("Naviga5_last11100_pratyush.json"))
with open("jayesh_id_21000.txt", 'rb') as filehandle:
    list_1 = pickle.load(filehandle)

In [49]:
list_1[0]

{'matching_results': 2,
 'session_token': '1_c09L6IWehUakyVl3d0imeTCxrj',
 'passages': [],
 'results': [{'id': '453767b1-2a6e-4f6b-9ba2-438fbd6a7ba2',
   'result_metadata': {'confidence': 0, 'score': 0},
   'copyright': '(c) 2020 inm. All Rights Reserved.',
   'geoOrigin': 'Belfast, Northern Ireland, United Kingdom',
   'sourceType': 'Newspaper',
   'source': 'Belfast Telegraph (Northern Ireland)',
   'corpus_document_id': '85851332710865434673',
   'description': 'MADELEINE McCann could still be alive, according to a former RUC detective who played a central role in the investigation into her disappearance. Jim Gamble said that "until there is evidence that she is dead, I\'m not going to accept it" after German prosecutors said a child sex predator may have killed the little girl. Full story, see pages 2-4.',
   'serviceName': 'Newspapers',
   'locationCode': [],
   'sourceClass': 'General News',
   'subjectCode': [],
   'text': 'MADELEINE McCann could still be alive, according to a f

In [50]:
len(list_1)

21100

# Extracting entities from html

In [51]:
# result = []
offset = 0
count = 1

In [52]:
for i in range(21100):
    res={}
    
    res["id"]= list_1[i]['results'][0]['id'] 
    res["doc_id"]= list_1[i]['results'][0]['corpus_document_id'] 

    source = list_1[i]['results'][0]['html']
    batRegex = re.compile(r'(.<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')

    lists = re.findall(batRegex,source)
    len_ = len(lists)
    for i in range(len_):
        lists[i] = list(lists[i])

    regex = re.compile(r'\svalue="(.*?)"')
    subst = ''
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

    regex = re.compile(r'\sidsrc="(.*?)"')
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

    regex = re.compile(r'\shref="(.*?)"')
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)


    for i in range(len_):
        lists[i][0]=lists[i][0].rsplit('/',1)
        lists[i][0][1]=lists[i][0][1].rsplit('>',1)
        keyword = lists[i][0][1][0]
        value = lists[i][1]

        if keyword in res:
            if value not in res[keyword]:
                res[keyword].append(value)       
        else:
            res[keyword] = [value]

    result.append(res)

In [53]:
len(result)

63407

In [54]:
result

[{'id': '4ffe7d4e-967a-47cb-95f4-2028276dcf30',
  'doc_id': '69699480789673276861',
  'location': ['GERMANY'],
  'chron': ['6/16/2020'],
  'p': [' : Not Specified']},
 {'id': '387b4ad0-a7f4-466a-8f26-061d556f0c61',
  'doc_id': '69700208441050923166',
  'org': ['St. Augustine Alligator Farm',
   'First Coast News',
   'Alligator Farm'],
  'person': ['Jim Darlington', 'John Brueggen']},
 {'id': '2d54cc17-6cfd-4d3b-af36-26d5c8777b16',
  'doc_id': '69703077675294679224',
  'person': ['Lorna Eneix'],
  'chron': ['June 1, 2020'],
  'org': ['Sharp Funeral Homes'],
  'a': ['www.sharpfuneralhomes.com']},
 {'id': 'e273292f-8fc0-46e3-a96c-e7b5eb740f6b',
  'doc_id': '69705439502282971339',
  'location': ['AURORA',
   'Aurora',
   'Louisburg',
   'Portsmouth, Ohio',
   '804 N. Bickett Blvd.',
   'Louisburg, NC'],
  'chron': ['Wednesday, May 27, 2020'],
  'person': ['Jennie Covert Smith',
   'William E. Ball',
   'Carolyn Gray',
   'Edgar Smith',
   'Betty Jane Hamilton',
   'William F. Ball',
   'C

In [57]:
del(list_1)

NameError: name 'list_1' is not defined

In [56]:
json.dump(result,open("Norm_EntitiesAll_Naviga5.json",'w'))

# Extracting person, org, location

In [2]:
result = json.load(open("Norm_EntitiesAll_Naviga5.json"))

In [3]:
result

[{'id': '4ffe7d4e-967a-47cb-95f4-2028276dcf30',
  'doc_id': '69699480789673276861',
  'location': ['GERMANY'],
  'chron': ['6/16/2020'],
  'p': [' : Not Specified']},
 {'id': '387b4ad0-a7f4-466a-8f26-061d556f0c61',
  'doc_id': '69700208441050923166',
  'org': ['St. Augustine Alligator Farm',
   'First Coast News',
   'Alligator Farm'],
  'person': ['Jim Darlington', 'John Brueggen']},
 {'id': '2d54cc17-6cfd-4d3b-af36-26d5c8777b16',
  'doc_id': '69703077675294679224',
  'person': ['Lorna Eneix'],
  'chron': ['June 1, 2020'],
  'org': ['Sharp Funeral Homes'],
  'a': ['www.sharpfuneralhomes.com']},
 {'id': 'e273292f-8fc0-46e3-a96c-e7b5eb740f6b',
  'doc_id': '69705439502282971339',
  'location': ['AURORA',
   'Aurora',
   'Louisburg',
   'Portsmouth, Ohio',
   '804 N. Bickett Blvd.',
   'Louisburg, NC'],
  'chron': ['Wednesday, May 27, 2020'],
  'person': ['Jennie Covert Smith',
   'William E. Ball',
   'Carolyn Gray',
   'Edgar Smith',
   'Betty Jane Hamilton',
   'William F. Ball',
   'C

In [4]:
dic = {'person':[],'org':[],'location':[],'person_id':[],'org_id':[],'location_id':[], 'person_docid':[], 'org_docid':[], 'location_docid':[]}

In [5]:
req = ['person','org','location','person_id','org_id','location_id','person_docid','org_docid', 'location_docid']

In [6]:
for item in result:
    
    for key,value in item.items():
        if key in req:
            dic[key].extend(value)
        if key == 'person':
            for i in range(len(item['person'])):
                dic['person_id'].append(item['id'])
                dic['person_docid'].append(item['doc_id'])
        if key == 'org':
            for i in range(len(item['org'])):
                dic['org_id'].append(item['id'])
                dic['org_docid'].append(item['doc_id'])
        if key == 'location':
            for i in range(len(item['location'])):
                dic['location_id'].append(item['id'])
                dic['location_docid'].append(item['doc_id'])

In [7]:
len(dic["person"])

233943

In [8]:
len(dic["org"])

189567

In [9]:
len(dic["location"])

238838

In [10]:
len(dic["person_id"])

233943

# Set (Remove same entities)

In [74]:
new_dict = {'person':[],'org':[],'location':[],'person_id':[],'org_id':[],'location_id':[], 'person_docid':[], 'org_docid':[], 'location_docid':[]}

In [75]:
per_set = set()
for i in range(len(dic["person"])):
    if dic["person"][i] not in per_set:
        per_set.add(dic["person"][i])
        new_dict["person"].append(dic["person"][i])
        new_dict["person_id"].append(dic["person_id"][i])
        new_dict["person_docid"].append(dic["person_docid"][i])

In [77]:
print(len(new_dict["person"]),len(new_dict["person_id"]),len(new_dict["person_docid"]))

163467 163467 163467


In [78]:
dic["person"] = set(dic["person"])
dic["person"] = list(dic["person"])
len(dic["person"])

163467

In [79]:
org_set = set()
for i in range(len(dic["org"])):
    if dic["org"][i] not in org_set:
        org_set.add(dic["org"][i])
        new_dict["org"].append(dic["org"][i])
        new_dict["org_id"].append(dic["org_id"][i])
        new_dict["org_docid"].append(dic["org_docid"][i])

In [80]:
print(len(new_dict["org"]),len(new_dict["org_id"]),len(new_dict["org_docid"]))

89463 89463 89463


In [81]:
loc_set = set()
for i in range(len(dic["location"])):
    if dic["location"][i] not in loc_set:
        loc_set.add(dic["location"][i])
        new_dict["location"].append(dic["location"][i])
        new_dict["location_id"].append(dic["location_id"][i])
        new_dict["location_docid"].append(dic["location_docid"][i])

In [82]:
print(len(new_dict["location"]),len(new_dict["location_id"]),len(new_dict["location_docid"]))

62741 62741 62741


# Creating Dataset

In [11]:
nav = pd.read_excel("Naviga5_Entity_Extraction_v01.xlsx")

In [12]:
nav = pd.DataFrame(nav)

In [13]:
df1 = pd.DataFrame(dic["person"],columns=["Person_Naviga"])
df2 = pd.DataFrame(dic["person_id"],columns=["Person_ID"])
df3 = pd.DataFrame(dic["person_docid"],columns=["Person_docID"])
df4 = pd.DataFrame(dic["org"],columns=["Company_Naviga"])
df5 = pd.DataFrame(dic["org_id"],columns=["Company_ID"])
df6 = pd.DataFrame(dic["org_docid"],columns=["Company_docID"])
df7 = pd.DataFrame(dic["location"],columns=["Location_Naviga"])
df8 = pd.DataFrame(dic["location_id"],columns=["Location_ID"])
df9 = pd.DataFrame(dic["location_docid"],columns=["Location_docID"])
df_doc = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9], ignore_index=True, axis=1)
nav = nav.append(df_doc)

In [14]:
nav.head()

,0,1,2,3,4,5,6,7,8
0,Jim Darlington,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,St. Augustine Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,GERMANY,4ffe7d4e-967a-47cb-95f4-2028276dcf30,69699480789673276861
1,John Brueggen,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,First Coast News,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,AURORA,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
2,Lorna Eneix,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,Aurora,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
3,Jennie Covert Smith,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Sharp Funeral Homes,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Louisburg,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
4,William E. Ball,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Lancaster Funeral and Cremation Services,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,"Portsmouth, Ohio",e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339


In [15]:
nav = nav.fillna("")
nav = nav.rename(columns={0: "Person_Naviga", 1: "Person_ID",2:"Person_docID", 3:"Company_Naviga", 4:"Company_ID", 5:"Company_docID", 6:"Location_Naviga",7:"Location_ID",8:"Location_docID"})

In [16]:
nav.head()

,Person_Naviga,Person_ID,Person_docID,Company_Naviga,Company_ID,Company_docID,Location_Naviga,Location_ID,Location_docID
0,Jim Darlington,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,St. Augustine Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,GERMANY,4ffe7d4e-967a-47cb-95f4-2028276dcf30,69699480789673276861
1,John Brueggen,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,First Coast News,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,AURORA,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
2,Lorna Eneix,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,Aurora,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
3,Jennie Covert Smith,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Sharp Funeral Homes,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Louisburg,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
4,William E. Ball,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Lancaster Funeral and Cremation Services,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,"Portsmouth, Ohio",e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339


In [17]:
nav.columns

Index(['Person_Naviga', 'Person_ID', 'Person_docID', 'Company_Naviga',
       'Company_ID', 'Company_docID', 'Location_Naviga', 'Location_ID',
       'Location_docID'],
      dtype='object')

In [18]:
for i in range(len(nav["Person_Naviga"])):
    lists = nav.loc[i,"Person_Naviga"].rsplit('>',1)
    if(len(lists)>1):
        nav.loc[i,"Person_Naviga"] = lists[1]

In [19]:
for i in range(len(nav["Company_Naviga"])):
    lists = nav.loc[i,"Company_Naviga"].rsplit('>',1)
    if(len(lists)>1):
        nav.loc[i,"Company_Naviga"] = lists[1]

In [20]:
for i in range(len(nav["Location_Naviga"])):
    lists = nav.loc[i,"Location_Naviga"].rsplit('>',1)
    if(len(lists)>1):
        nav.loc[i,"Location_Naviga"] = lists[1]

In [21]:
nav.head()

,Person_Naviga,Person_ID,Person_docID,Company_Naviga,Company_ID,Company_docID,Location_Naviga,Location_ID,Location_docID
0,Jim Darlington,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,St. Augustine Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,GERMANY,4ffe7d4e-967a-47cb-95f4-2028276dcf30,69699480789673276861
1,John Brueggen,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,First Coast News,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,AURORA,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
2,Lorna Eneix,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Alligator Farm,387b4ad0-a7f4-466a-8f26-061d556f0c61,69700208441050923166,Aurora,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
3,Jennie Covert Smith,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Sharp Funeral Homes,2d54cc17-6cfd-4d3b-af36-26d5c8777b16,69703077675294679224,Louisburg,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339
4,William E. Ball,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,Lancaster Funeral and Cremation Services,e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339,"Portsmouth, Ohio",e273292f-8fc0-46e3-a96c-e7b5eb740f6b,69705439502282971339


In [22]:
nav.to_excel("Naviga5_Entity_Extraction_v01.xlsx", sheet_name="Naviga",index=False)

# Combining ID's and docID's

### of same entities from different docs

In [23]:
nav = pd.read_excel("Naviga5_Entity_Extraction_v01.xlsx", sheet_name="Naviga")

In [24]:
Person=[]
Person_ID=[]
Person_docID=[]

In [25]:
i=0
for l in [Person,Person_ID,Person_docID]:
        l.extend(nav.iloc[:,i].dropna().values)
        i+=1

In [26]:
len(Person)

233943

In [27]:
uni_Person=dict()
for i in range(len(Person)):
    if Person[i] in uni_Person:
        uni_Person[Person[i]].append(Person_ID[i])
    else:
        uni_Person[Person[i]]=[]
        uni_Person[Person[i]].append(Person_ID[i])
Person_ID=list((uni_Person.values()))

uni_Person=dict()
for i in range(len(Person)):
    if Person[i] in uni_Person:
        uni_Person[Person[i]].append(Person_docID[i])
    else:
        uni_Person[Person[i]]=[]
        uni_Person[Person[i]].append(Person_docID[i])
Person_docID=list((uni_Person.values()))
Person=list((uni_Person.keys()))

In [28]:
Company=[]
Company_ID=[]
Company_docID=[]

In [29]:
i=3
for l in [Company,Company_ID,Company_docID]:
    l.extend(nav.iloc[:,i].dropna().values)
    i+=1

In [30]:
len(Company)

189567

In [31]:
uni_Company=dict()
for i in range(len(Company)):
    if Company[i] in uni_Company:
        uni_Company[Company[i]].append(Company_ID[i])
    else:
        uni_Company[Company[i]]=[]
        uni_Company[Company[i]].append(Company_ID[i])
Company_ID=list((uni_Company.values()))

uni_Company=dict()
for i in range(len(Company)):
    if Company[i] in uni_Company:
        uni_Company[Company[i]].append(Company_docID[i])
    else:
        uni_Company[Company[i]]=[]
        uni_Company[Company[i]].append(Company_docID[i])
Company_docID=list((uni_Company.values()))
Company=list((uni_Company.keys()))

In [32]:
len(Company)

83923

In [33]:
Location=[]
Location_ID=[]
Location_docID=[]

In [34]:
i=6
for l in [Location,Location_ID,Location_docID]:
    l.extend(nav.iloc[:,i].dropna().values)
    i+=1

In [35]:
len(Location)

238838

In [36]:
uni_Location=dict()
for i in range(len(Location)):
    if Location[i] in uni_Location:
        uni_Location[Location[i]].append(Location_ID[i])
    else:
        uni_Location[Location[i]]=[]
        uni_Location[Location[i]].append(Location_ID[i])
Location_ID=list((uni_Location.values()))

uni_Location=dict()
for i in range(len(Location)):
    if Location[i] in uni_Location:
        uni_Location[Location[i]].append(Location_docID[i])
    else:
        uni_Location[Location[i]]=[]
        uni_Location[Location[i]].append(Location_docID[i])
Location_docID=list((uni_Location.values()))
Location=list((uni_Location.keys()))

In [37]:
len(Location)

56592

In [38]:
df=pd.DataFrame({'Person': pd.Series(Person), 'Person_ID': pd.Series(Person_ID), 'Person_docID': pd.Series(Person_docID),'Company': pd.Series(Company), 'Company_ID': pd.Series(Company_ID), 'Company_docID': pd.Series(Company_docID), 'Location': pd.Series(Location), 'Location_ID': pd.Series(Location_ID), 'Location_docID': pd.Series(Location_docID)})

In [39]:
df

,Person,Person_ID,Person_docID,Company,Company_ID,Company_docID,Location,Location_ID,Location_docID
0,Jim Darlington,"[387b4ad0-a7f4-466a-8f26-061d556f0c61, 0c0f378...","[69700208441050923166, 70877986393294968926]",St. Augustine Alligator Farm,"[387b4ad0-a7f4-466a-8f26-061d556f0c61, 0c0f378...","[69700208441050923166, 70877986393294968926]",GERMANY,"[4ffe7d4e-967a-47cb-95f4-2028276dcf30, 776c3e1...","[69699480789673276861, 69706528919933456921, 6..."
1,John Brueggen,"[387b4ad0-a7f4-466a-8f26-061d556f0c61, 0c0f378...","[69700208441050923166, 70877986393294968926]",First Coast News,"[387b4ad0-a7f4-466a-8f26-061d556f0c61, 0c0f378...","[69700208441050923166, 70877986393294968926, 9...",AURORA,[e273292f-8fc0-46e3-a96c-e7b5eb740f6b],[69705439502282971339]
2,Lorna Eneix,[2d54cc17-6cfd-4d3b-af36-26d5c8777b16],[69703077675294679224],Alligator Farm,"[387b4ad0-a7f4-466a-8f26-061d556f0c61, 0c0f378...","[69700208441050923166, 70877986393294968926]",Aurora,"[e273292f-8fc0-46e3-a96c-e7b5eb740f6b, 63c8c44...","[69705439502282971339, 70279708927980536461, 7..."
3,Jennie Covert Smith,[e273292f-8fc0-46e3-a96c-e7b5eb740f6b],[69705439502282971339],Sharp Funeral Homes,"[2d54cc17-6cfd-4d3b-af36-26d5c8777b16, 55f9a1b...","[69703077675294679224, 78306168391917008073, 8...",Louisburg,"[e273292f-8fc0-46e3-a96c-e7b5eb740f6b, e4b5e15...","[69705439502282971339, 75560921580039928022, 5..."
4,William E. Ball,[e273292f-8fc0-46e3-a96c-e7b5eb740f6b],[69705439502282971339],Lancaster Funeral and Cremation Services,[e273292f-8fc0-46e3-a96c-e7b5eb740f6b],[69705439502282971339],"Portsmouth, Ohio","[e273292f-8fc0-46e3-a96c-e7b5eb740f6b, e341afe...","[69705439502282971339, 85254659789963008057, 6..."
...,...,...,...,...,...,...,...,...,...
160696,John Shannon,[04ee1de7-c20e-4966-a785-fa0310099519],[39795561005387614954],NaN,NaN,NaN,NaN,NaN,NaN
160697,Charles Jordan,[04ee1de7-c20e-4966-a785-fa0310099519],[39795561005387614954],NaN,NaN,NaN,NaN,NaN,NaN
160698,L.C. Blocker,[04ee1de7-c20e-4966-a785-fa0310099519],[39795561005387614954],NaN,NaN,NaN,NaN,NaN,NaN
160699,AMANDA BOSHEARS,[17f93ae2-2737-4d2e-81eb-947a1d868a8d],[39799627168097113012],NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.to_excel("Naviga5_Entity_Extraction_v02.xlsx",index=False)